# Homework 2

In [1]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2018 term"

This homework covers material from the sentiment classification unit. The primary value of doing the work is that it provides more hands-on experience with the dataset and the models we explored. All the code you write has potential value in the bake-off for this unit as well.

Submission URL: https://canvas.stanford.edu/courses/83399/quizzes/50657

In [54]:
from collections import Counter
from sklearn.linear_model import LogisticRegression
import sklearn.naive_bayes
import sklearn.model_selection
import scipy.stats
from sgd_classifier import BasicSGDClassifier
# from tf_shallow_neural_classifier import TfShallowNeuralClassifier
import sst
import utils

## Questions 1–4: Reproducing a Socher et al's NaiveBayes baseline [4 points]

[Socher et al. 2013](http://www.aclweb.org/anthology/D/D13/D13-1170.pdf) compare against a Naive Bayes baseline with bigram features. See how close you can come to reproducing the performance of this model on the binary, root-only problem (values in the rightmost column of their Table 1, rows 1 and 3).

Specific tasks:

1. Write a bigrams feature function, on the model of `unigrams_phi`. Call this `bigrams_phi`. In writing this function, ensure that each example is padded with start and end symbols (say, `<S>` and `</S>`), so that these contexts are properly reflected in the feature space.

1. Write a function `fit_nb_classifier_with_crossvalidation` that serves as a wrapper for `sklearn.naive_bayes.MultinomialNB` and searches over these values for the smoothing parameter `alpha`: `[0.1, 0.5, 1.0, 2.0]`, using 3-fold cross-validation.

1. Use `sst.experiment` to run the experiments, assessing against `dev_reader`.

__To submit:__

1. Your `bigrams_phi`
1. Your `fit_nb_classifier`
1. Your call to `sst.experiment` 
1. The average F1 score that `sst.experiment` reported.

__A note on performance__: in our experience, the bigrams Naive Bayes model gets around 0.75. It's fine to submit answers with comparable numbers; the Socher et al. baselines are very strong. We're not evaluating how good your model is; we want to see your code, and we're interested to see what the range of F1 scores is across the whole class.

In [45]:
def bigrams_phi(tree):
    """The basis for a unigrams feature function.
    
    Parameters
    ----------
    tree : nltk.tree
        The tree to represent.
    
    Returns
    -------    
    defaultdict
        A map from strings to their counts in `tree`. (Counter maps a 
        list to a dict of counts of the elements in that list.)
    
    """
    
    text = tree.leaves()
    text = ['<S>'] + text + ['</S>']
    bigrams = []
    for i in range(len(text)-1):
        bigrams.append((text[i], text[i+1]))
    
    return Counter(bigrams)

In [62]:
def fit_nb_classifier_with_crossvalidation(X, y, a = 0.5):

    basemod = sklearn.naive_bayes.MultinomialNB()
    cv = 3
    param_grid = {'alpha': [0.1, 0.5, 1.0, 2.0]}    
    return sst.fit_classifier_with_crossvalidation(X, y, basemod, cv, param_grid)

In [64]:
_ = sst.experiment(
    bigrams_phi,
    fit_nb_classifier_with_crossvalidation,
    train_reader=sst.train_reader, 
    assess_reader=sst.dev_reader,
    verbose=True)

Best params {'alpha': 0.5}
Best score: 0.725
Accuracy: 0.748
             precision    recall  f1-score   support

   negative      0.761     0.708     0.734       428
   positive      0.736     0.786     0.760       444

avg / total      0.749     0.748     0.747       872



## Question 5–6: A more powerful vector-summing baseline [4 points]

In the section [Distributed representations as features](sst_03_neural_networks.ipynb#Distributed-representations-as-features), we looked at a baseline for the binary SST problem in which each example is modeled as the sum of its 50-dimensional GloVe representations. A `LogisticRegression` model was used for prediction. A neural network might do better here, since there might be complex relationships between the input feature dimensions that a linear classifier can't learn. 

To address this question, rerun the experiment with `tf_shallow_neural_classifier.TfShallowNeuralClassifier` as the classifier. Specs:
* Use `sst.experiment` to conduct the experiment. 
* Using 3-fold cross-validation, exhaustively explore this set of hyperparameter combinations:
  * The hidden dimensionality at 50, 100, and 200.
  * The hidden activation function as `tf.nn.tanh` or `tf.nn.relu`.
* (For all other parameters to `TfShallowNeuralClassifier`, use the defaults.)

__To submit:__

* Your average F1 score according to `sst.experiment`. 
* The optimal hyperparameters chosen in your experiment. (You can just paste in the dict that `sst._experiment` prints.)

No need to include your supporting code. 

We're not evaluating the quality of your model. (We've specified the protocols completely, but there will still be a  lot of variation in the results.) However, the primary goal of this question is to get you thinking more about this strikingly good baseline feature representation scheme for SST, so we're sort of hoping you feel compelled to try out variations on your own.

## Questions 7–8: Bidirectional RNN [2 points]

The auxiliary notebook `tensorflow_models.ipynb` [subclasses TfRNNClassifier with a bidirectional RNN](tensorflow_models.ipynb#A-bidirectional-RNN-Classifier). In this model, the RNN is run in both directions and the concatenation of the two final states is used as the basis for the classification decision. Evaluate this model against the SST dev set. You can set up the model however you wish for this.

__To submit:__

* Your call to `TfBidirectionalRNNClassifier` (so that we can see the hyperparmeters you chose).
* Your average F1 score according to a `classification_report` on the `dev` set.

As above, we will not evaluate you based on how good your F1 score is. You just need to submit one. __There is even value in seeing what really doesn't work__, so low scores have interest!